# Security & Privacy (CC2009) - 2023/2024

## ASSIGNMENT #1: Performance Benchmarking of Cryptographic Mechanisms

### Due date: March 22, 23:59
### Grading: Assignment #1 is worth 2 points

In this exercise you should measure the time AES, RSA and SHA take to process files of different sizes, using a
python implementation of the encryption/description and hash mechanisms.
Some notes:
- You should measure the time of cryptographic operations/algorithms only, not including the time for generation
of files and others side aspects.
- If you use padding, this may affect the results specially for small file sizes

-------------------------------------------------------------------------------------------------------------------------------------

A. Generate random text files with the following sizes:
- For AES (in bytes): 8, 64, 512, 4096, 32768, 262144, 2097152 
- For SHA (in bytes): 8, 64, 512, 4096, 32768, 262144, 2097152 
- For RSA (in bytes): 2, 4, 8, 16, 32, 64, 128


In [26]:
import os

# Sizes in bytes
file_sizes_aes = [8, 64, 512, 4096, 32768, 262144, 2097152]
file_sizes_sha = [8, 64, 512, 4096, 32768, 262144, 2097152]
file_sizes_rsa = [2, 4, 8, 16, 32, 64, 128]

def generate_random_text_file(size_bytes):
    return os.urandom(size_bytes)

# Generate random text files for aes
for size in file_sizes_aes:
    with open(f"aes_{size}.txt", "wb") as f:
        f.write(generate_random_text_file(size))

# Generate random text files for sha
for size in file_sizes_sha:
    with open(f"sha_{size}.txt", "wb") as f:
        f.write(generate_random_text_file(size))

# Generate random text files for rsa
for size in file_sizes_rsa:
    with open(f"rsa{size}.txt", "wb") as f:
        f.write(generate_random_text_file(size))

-------------------------------------------------------------------------------------------------------------------------------------

B. Encrypt and decrypt all these files using AES. Employ a key of 256 bits. Measure the time it takes to encrypt
and decrypt each of the files. To do this, you might want to use the python module timeit.
Make sure to produce statistically significant results. Do results change if you run a fixed algorithm over
the same file multiple times? And what if you run an algorithm over multiple randomly generated files
of fixed size?

In [27]:
import os
import random
import string
import timeit
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend


def encrypt_file_aes(input_file, output_file, key):
    with open(input_file, 'rb') as f:
        plaintext = f.read()
    padded_plaintext = pad(plaintext, algorithms.AES.block_size)
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    encrypted_data = encryptor.update(padded_plaintext) + encryptor.finalize()
    with open(output_file, 'wb') as f:
        f.write(encrypted_data)

def decrypt_file_aes(input_file, output_file, key):
    with open(input_file, 'rb') as f:
        encrypted_data = f.read()
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_data = decryptor.update(encrypted_data) + decryptor.finalize()
    with open(output_file, 'wb') as f:
        f.write(decrypted_data.rstrip(b'\x00'))

def pad(data, block_size):
    padding_length = block_size - (len(data) % block_size)
    return data + bytes([padding_length] * padding_length)

# Key for AES encryption and decryption (256-bit key)
key = os.urandom(32) 

# Dictionary to store encryption and decryption times for each file size
encryption_times = {}
decryption_times = {}

# Repeat each operation multiple times to get statistically significant results
num_runs = 10

# Encrypt and decrypt each file size
for size in aes_sizes:
    input_file = f"aes_{size}.txt"
    encrypted_file = f"aes_{size}_encrypted.txt"
    decrypted_file = f"aes_{size}_decrypted.txt"
        
    # Measure encryption time
    encryption_time = timeit.timeit(lambda: encrypt_file_aes(input_file, encrypted_file, key), number=num_runs)
    encryption_times[size] = encryption_time / num_runs
    
    # Measure decryption time
    decryption_time = timeit.timeit(lambda: decrypt_file_aes(encrypted_file, decrypted_file, key), number=num_runs)
    decryption_times[size] = decryption_time / num_runs
    
    # Delete generated files
    os.remove(input_file)
    os.remove(encrypted_file)
    os.remove(decrypted_file)

# Display encryption and decryption times
print("Encryption times (in seconds):")
for size, time in encryption_times.items():
    print(f"AES {size} bytes: {time:.6f}")
    
print("\nDecryption times (in seconds):")
for size, time in decryption_times.items():
    print(f"AES {size} bytes: {time:.6f}")


Encryption times (in seconds):
AES 8 bytes: 0.000526
AES 64 bytes: 0.000250
AES 512 bytes: 0.000103
AES 4096 bytes: 0.000170
AES 32768 bytes: 0.000135
AES 262144 bytes: 0.000194
AES 2097152 bytes: 0.001787

Decryption times (in seconds):
AES 8 bytes: 0.000203
AES 64 bytes: 0.000113
AES 512 bytes: 0.000098
AES 4096 bytes: 0.000164
AES 32768 bytes: 0.000103
AES 262144 bytes: 0.000174
AES 2097152 bytes: 0.001443


-------------------------------------------------------------------------------------------------------------------------------------

C. Using the python module for RSA encryption and decryption, measure the time of RSA encryption and decryption
for the file sizes listed in part A, with a key of size 2048 bits (minimum recommended for RSA).

-------------------------------------------------------------------------------------------------------------------------------------

D. Measure the time for SHA-256 hash generation for the file sizes listed in part a.

In [28]:
import time
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

def time_aes(file_sizes):
    backend = default_backend()
    key = os.urandom(32)  # 256-bit key
    iv = os.urandom(16)   # 128-bit IV
    times = []
    for size in file_sizes:
        data = generate_random_text_file(size)
        start_time = time.time()
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=backend)
        encryptor = cipher.encryptor()
        ciphertext = encryptor.update(data) + encryptor.finalize()
        decryptor = cipher.decryptor()
        plaintext = decryptor.update(ciphertext) + decryptor.finalize()
        end_time = time.time()
        times.append(end_time - start_time)
    return times

def time_sha(file_sizes):
    times = []
    for size in file_sizes:
        data = generate_random_text_file(size)
        start_time = time.time()
        hash_obj = hashes.Hash(hashes.SHA256())
        hash_obj.update(data)
        hash_result = hash_obj.finalize()
        end_time = time.time()
        times.append(end_time - start_time)
    return times

def time_rsa(file_sizes):
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    times = []
    for size in file_sizes:
        data = generate_random_text_file(size)
        start_time = time.time()
        ciphertext = public_key.encrypt(
            data,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        plaintext = private_key.decrypt(
            ciphertext,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        end_time = time.time()
        times.append(end_time - start_time)
    return times


print("AES Times:", time_aes(file_sizes_aes))
print("SHA Times:", time_sha(file_sizes_sha))
print("RSA Times:", time_rsa(file_sizes_rsa))


AES Times: [0.00012087821960449219, 3.5762786865234375e-05, 5.817413330078125e-05, 0.00013303756713867188, 0.0004799365997314453, 0.0006990432739257812, 0.00762176513671875]
SHA Times: [3.62396240234375e-05, 5.245208740234375e-06, 3.814697265625e-06, 5.0067901611328125e-06, 1.6927719116210938e-05, 0.00013399124145507812, 0.0009448528289794922]
RSA Times: [0.0023071765899658203, 0.0010769367218017578, 0.0010671615600585938, 0.0010769367218017578, 0.0010650157928466797, 0.0011157989501953125, 0.0011379718780517578]


-------------------------------------------------------------------------------------------------------------------------------------

E. Prepare a report of your observations, including the following information:
- Code implemented for points b., c., and d. above
- Brief explanation of the main components of the code (the rest should be submitted in a separate compressed
file)
- Explain how you generated/obtained the results – must be statistically significant. This must include a
description of the experimental setup (e.g. computer characteristics, OS, software versions).
- Plots showing: (i) AES encryption/decryption times; (ii) RSA encryption times; (iii) RSA decryption times;
and (iv) SHA digests generation times (plots can be combined for easier comparison). In these graphs, the
X axis should plot the file sizes in units of bytes, and the Y axis should plot time measurements in units of
microseconds (us).
1
- The report should also analyze and explain the performance results of:
    - Comparison between AES encryption and RSA encryption.
    - Comparison between AES encryption and SHA digest generation.
    - Comparison between RSA encryption and decryption times.